<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [4]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [8]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [10]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation,LeakyReLU
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [0]:
# 3位以内にマークを付ける
ldat["high_rank"] = 0
ldat.loc[ldat["rank"] <= 3, "high_rank"] = 1

In [0]:
# 人気3位以内にマークを付ける
ldat["high_prev_popularity"] = 0
ldat.loc[ldat["prev_popularity"] <= 3, "high_prev_popularity"] = 1

In [0]:
# スピードを出す
ldat["prev_speed"] = ldat["prev_distance"] / ldat["prev_time_float"] 

In [0]:
# スピードが早い馬（平均より早い）にマークを付ける
mean = ldat["prev_speed"].mean() 

In [0]:
ldat["prev_speed_high"] =  (ldat["prev_speed"] > mean).astype(int)

In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("high_rank == 1")

In [0]:
loser = sdat.query("high_rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1).reset_index(drop=True)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
x = sampled[['high_prev_popularity',"speed_high"]].dropna(0)

In [0]:
x = sampled[['high_prev_popularity']].dropna(0)

In [0]:
#y = sampled[["high_rank"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["high_rank"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"])

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
testorg = sampled.tail(len(x)-trainlen) 

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 20


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(256, activation=None))
model.add(LeakyReLU(alpha=0.3))

model.add(Dense(128, activation=None))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [135]:
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 512)               1536      
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               32896     
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 2)                

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [139]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 112774 samples, validate on 112774 samples
Epoch 1/20
 91136/112774 [=======================>......] - ETA: 1s - loss: 0.6623 - accuracy: 0.6085

KeyboardInterrupt: ignored

In [0]:
predictions = model.predict(testx)

In [132]:
print(predictions)

[[0.5588973  0.4401093 ]
 [0.5588973  0.4401093 ]
 [0.5728635  0.42577553]
 ...
 [0.27815038 0.7217264 ]
 [0.5728635  0.42577553]
 [0.5728635  0.42577553]]


In [46]:
score = model.evaluate(testx, testy, verbose=1)


112774/112774 [==============================] - 4s 32us/step


In [47]:

print("Test score", score[0])
print("Test accuracy", score[1])

Test score 0.6650966940119041
Test accuracy 0.6069040894508362


In [0]:
predict = pd.DataFrame(predictions, columns={"lose","win"})

In [0]:
result =  pd.concat([ predict.reset_index(),testorg.reset_index()],axis=1)

In [0]:
result["final_return"] = result["return"]
result.loc[result["return"] == 0, "final_return"] = -1


In [51]:
result.query("win > 0.5").sum()["final_return"]

20519.5

In [52]:
result.query("win > 0.5").mean()["final_return"]

0.692244113082786

In [53]:
result.count()

index                    112774
lose                     112774
win                      112774
index                    112774
rank                     112120
                          ...  
start_time_float         112774
prev_start_time_float    112774
high_rank                112774
high_prev_popularity     112774
final_return             112774
Length: 66, dtype: int64

In [54]:
testorg.sum()["return"]

197487.19999999998

In [55]:
testorg.count()

rank                     112120
frame_num                112774
horse_num                112774
name                     112774
sex                      112774
                          ...  
prev_time_float          112774
start_time_float         112774
prev_start_time_float    112774
high_rank                112774
high_prev_popularity     112774
Length: 61, dtype: int64